<a href="https://colab.research.google.com/github/danny790513/data-course-sample/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-25 09:49:30--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.3’

All_Beauty.csv.3    100%[===================>]  14.78M  43.8MB/s    in 0.3s    

2021-12-25 09:49:30 (43.8 MB/s) - ‘All_Beauty.csv.3’ saved [15499476/15499476]

--2021-12-25 09:49:30--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.3’

meta_All_Beauty.jso 100%[===================>]   9.85M  32.7MB/s    in 0.3s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [8]:
import datetime
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    # 以近期的資料當作訓練資料, 大概1個月~1季內會比較有參考價值
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = 30)) > '2018-09-01']
    # 以商品分組計算平均評分&購買數量, 並計算熱銷商品排行
    grouped = training_data.groupby('asin').agg({'overall': 'mean', 'reviewerID': 'count'}).rename(columns={'overall': 'avg_score', 'reviewerID': 'sales_count'}).reset_index()
    top_selling = grouped.sort_values(by=['sales_count', 'avg_score'], ascending=False, ignore_index=True)
    # 開始對每個user推薦商品
    for user in users:
        recommend_k = []
        if user in training_data['reviewerID'].values:  # 若user曾在訓練期間內買過東西，就推薦過去買過的商品中的also buy或also view給他們參考
            record_df = training_data.loc[training_data['reviewerID'] == user].sort_values(by='DATE', ascending=False, ignore_index=True)
            for record in record_df['asin'].values:
                meta = metadata.loc[metadata['asin'] == record]
                recommend_k += meta['also_buy'].tolist()[0]
                recommend_k += meta['also_view'].tolist()[0]
        recommend_k += top_selling['asin'].head(k).tolist()  # 沒買過的user或推薦商品數不足k的用熱銷商品補足
        recommendations[user] = recommend_k[:k]
    return recommendations
ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user

## 結果評估

In [9]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.1576271186440678

In [11]:
'''以下為其他效果不太顯著的作法'''
def recommender_dev(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    grouped = training_data.groupby('asin').agg({'overall': 'mean', 'reviewerID': 'count'}).rename(columns={'overall': 'avg_score', 'reviewerID': 'sales_count'}).reset_index()
    recommendations = {}
    # 用最多評論的商品(最熱銷)當作推薦清單 -> 0.08305084745762711
    top_selling = grouped.sort_values(by=['sales_count', 'avg_score'], ascending=False, ignore_index=True)
    recommendations = {user: top_selling['asin'].head(k).tolist() for user in users}

    # 單純用平均評論最高的商品當作推薦清單 -> 0.0
    best_ratings = grouped.sort_values(by=['avg_score', 'sales_count'], ascending=False, ignore_index=True)
    recommendations = {user: best_ratings['asin'].head(k).tolist() for user in users}

    # 對評論數量做限制後才對平均評論商品進行排序 -> 100:0.003389830508474576, 500:0.0, 1000:0.0, 2000:0.08305084745762711(剛好是最熱銷的top10) 1500:0.0
    best_ratings = grouped.loc[grouped['sales_count']>2000].sort_values(by=['avg_score', 'sales_count'], ascending=False, ignore_index=True)
    recommendations = {user: best_ratings['asin'].head(k).tolist() for user in users}

    # 從metadata中以rule提取推薦商品資料
    # 以rank排序後推薦前K名 -> 0.005084745762711864
    meta_df = metadata.copy()
    meta_df['rank'] = meta_df['rank'].str.split(expand = True)[0]
    meta_df['rank'] = pd.to_numeric(meta_df['rank'].str.replace(',', ''))
    meta_df = meta_df.sort_values(by=['rank'],ignore_index=True)
    recommendations = {user: meta_df['asin'].head(k).tolist() for user in users}

    # 以price排序後推薦最便宜的 -> 0.0
    meta_df['price'] = meta_df['price'].str.findall(r'\d+.\d+')
    meta_df['price'] = meta_df['price'].apply(lambda x: x[0] if x else None)
    meta_df['price'] = pd.to_numeric(meta_df['price'].str.replace(',', ''))
    meta_df = meta_df.sort_values(by=['price', 'rank'], ascending=False, ignore_index=True)
    recommendations = {user: meta_df['asin'].head(k).tolist() for user in users}
    return recommendations
ratings_by_user = recommender_dev(ratings_trainings, users)